# Klassifikation der Anfänge von Kurzgeschichten nach ihrem Informationsgrad

In [1]:
pip install anthropic


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import anthropic

In [3]:
pip install python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd

# Loading Data

In [5]:
file_path = 'Kurzgeschichten_Anfaenge_DHD_2025.xlsx'

In [6]:
df = pd.read_excel(file_path)

In [7]:
df.head()

,Autor,Titel,Absatz,Eingangssatz,Person,Gegenstand,Ort,Zeit,Score,Klassifikation
0,Böll,Der Dieb,"Am meisten war ich darüber erstaunt, wie schne...","Am meisten war ich darüber erstaunt, wie schne...",1,1,0,0,2,sehr wenig Information
1,Böll,Aus der »Vorzeit«,Nachdem die Kompanie vom Kasernenhof abgerückt...,Nachdem die Kompanie vom Kasernenhof abgerückt...,1,2,2,0,5,wenig Information
2,Böll,Im Käfig,Irgendeiner stand am Zaun und blickte grübelnd...,Irgendeiner stand am Zaun und blickte grübelnd...,1,2,2,0,5,wenig Information
3,Böll,In guter Hut,"Als ich wach wurde, wollte ich es zuerst gar n...","Als ich wach wurde, wollte ich es zuerst gar n...",1,2,0,0,3,sehr wenig Information
4,Böll,Todesursache: Hakennase,Als der Leutnant Hegemüller in sein Quartier z...,Als der Leutnant Hegemüller in sein Quartier z...,2,2,2,0,6,wenig Information


In [8]:
df.Klassifikation.value_counts()

Klassifikation
sehr wenig Information    25
wenig Information         22
viel Information           8
Name: count, dtype: int64

## Claude SetUp

In [9]:
client = anthropic.Anthropic(
    api_key = os.getenv('ANTHROPIC_API_KEY')
)

# Evaluating

In [10]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

In [11]:
def get_completion_ant(prompt):
    message = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1024,
        messages=[
            {"role": "user", "content": prompt}
        ])
    return message.content[0].text  

In [12]:
results = []
for text in df.Eingangssatz:
    snippet = text
    
    prompt = f"""
    #Role
    You are a literary scholar
    
    #Instruction
    Your task is to recognize points of indeterminacy in German-language narrative texts given to you. The opening of a literary narrative has a point of indeterminacy precisely when it is characterized by a lack of information regarding the central character, the subject, the place or the time of the plot.
    The degree of lack of information is determined using the following scorin system: The starting point is information about the character, subject, place and time of the narrative:
    If one or more of these dimensions are represented by pronouns or proper nouns, 1 point is awarded for each.
    If they are represented by definite noun phrases with determiners, 2 points are awarded for each.
    If they are represented by indefinite noun phrases, 3 points are awarded for each.
    If the information on the character, subject, time and place of the story uses a word type other than those mentioned above, no points will be awarded!
    Points can only be awarded once for each of the 4 categories. This means that if there are two information carriers for the location, points are only awarded once for these!

    Give first an analysis, calculate the score and then select one of the following German labels to classify the degree of the lack of information regarding the central character, the subject, the place or the time of the plot in the given text snippet:
        If the total score of the sentence is between 10 and 12 points – Label: sehr viel Information
        If the total score of the sentence is between 7 and 9 points - Label: viel Information
        If the total score of the sentence is between 4 and 6 points - Label: wenig Information
        If the total score of the sentence is between 0 and 3 points - Label: sehr wenig Information
        
    
    #Output
    Use the following output format:
    Analysis: str
    Label: str
    
    #
    Analyze the following German text: {snippet}
    Analysis: 
    Label:
    """
  
    result = get_completion_ant(prompt)
    print(result)
    results.append(result)

Analysis: 
In this sentence, we can identify the following elements:

1. Character: "ich" (I) - represented by a pronoun (1 point)
2. Subject: "darüber" (about that) - represented by a pronoun (1 point)
3. Time: Not explicitly mentioned (0 points)
4. Place: Not mentioned (0 points)

The sentence focuses on the character's reaction (being astonished) and refers to something that happened quickly and easily, but it doesn't provide specific information about time or place. The use of pronouns for both the character and the subject of astonishment contributes to a sense of indeterminacy.

Total score: 1 + 1 = 2 points

Label: sehr wenig Information
Analysis: 
1. Character: "Renatus" (proper noun) - 1 point
2. Subject: "die Kompanie" (definite noun phrase) - 2 points
3. Place: "vom Kasernenhof" and "auf die Stube" (definite noun phrases) - 2 points (only counted once)
4. Time: "Nachdem" (temporal conjunction, not scored)

The sentence provides information about the character (Renatus), the 

In [13]:
results_1 = []
for text in results:
    if "stark unbestimmt" in text:
        results_1.append("stark unbestimmt")
    elif "wenig unbestimmt" in text:
        results_1.append("wenig unbestimmt")
    elif "nicht unbestimmt" in text:
        results_1.append("nicht unbestimmt")
    elif "unbestimmt" in text:
        results_1.append("unbestimmt")
    else:
        results_1.append("NaN")

In [14]:
results_1 = []
for text in results:
    if "sehr viel Information" in text:
        results_1.append("sehr viel Information")
    elif "viel Information" in text:
        results_1.append("viel Information")
    elif "sehr wenig Information" in text:
        results_1.append("sehr wenig Information")
    elif "wenig Information" in text:
        results_1.append("wenig Information")
    else:
        results_1.append("NaN")

In [15]:
results_se = pd.Series(results_1)

In [16]:
predictions = results_se
ground_truth = df.Klassifikation

In [17]:
pd.DataFrame(zip(ground_truth, predictions))

,0,1
0,sehr wenig Information,sehr wenig Information
1,wenig Information,wenig Information
2,wenig Information,wenig Information
3,sehr wenig Information,sehr wenig Information
4,wenig Information,sehr wenig Information
5,wenig Information,wenig Information
6,wenig Information,wenig Information
7,wenig Information,wenig Information
8,sehr wenig Information,wenig Information
9,sehr wenig Information,sehr wenig Information


In [18]:
pd.DataFrame(list(zip([f1_score(ground_truth, predictions, average="weighted")],
                      [recall_score(ground_truth, predictions,  average="weighted")],
                      [precision_score(ground_truth, predictions, average="weighted")],
                      [accuracy_score(ground_truth, predictions,)])),
                      columns = ["F1", "Recall", "Precision", "Accuracy"])

,F1,Recall,Precision,Accuracy
0,0.632176,0.636364,0.638865,0.636364


In [19]:
print(classification_report(ground_truth, predictions))

                        precision    recall  f1-score   support

sehr wenig Information       0.71      0.60      0.65        25
      viel Information       0.43      0.38      0.40         8
     wenig Information       0.63      0.77      0.69        22

              accuracy                           0.64        55
             macro avg       0.59      0.58      0.58        55
          weighted avg       0.64      0.64      0.63        55

